In [1]:
from cifar10_classifier import CIFAR10Classifier
import config

import torch
from torchvision import transforms

In [2]:
from utils.utils import set_seed, set_deterministic
set_seed(config.SEED)
set_deterministic(
    deterministic=config.DETERMINISTIC,
    benchmark=config.BENCHMARK
)

🧬 Setting seed: 42


In [3]:
from utils.data_utils import (
    compute_mean_std, get_transforms,
    load_cifar10_datasets, split_train_val, create_loaders,
    get_dataset_info
)

In [4]:
# Normalization
raw_dataset, _ = load_cifar10_datasets(transform=transforms.ToTensor())
mean, std = compute_mean_std(raw_dataset)

# Apply transformations
full_transform = get_transforms(mean, std)

# Load with transformations
train_dataset, test_dataset = load_cifar10_datasets(transform=full_transform)

# Split
train_subset, val_subset = split_train_val(train_dataset, split_ratio=config.SPLIT_RATIO)

# Loaders
train_loader, val_loader, test_loader = create_loaders(train_subset, val_subset, test_dataset, batch_size=config.BATCH_SIZE)


📥 Downloading/loading CIFAR-10 datasets...
Files already downloaded and verified
Files already downloaded and verified
✅ Loaded training samples: 50000, test samples: 10000
📊 Computing mean and std...
✅ Mean: [0.4913996756076813, 0.48215851187705994, 0.4465310275554657], Std: [0.24703219532966614, 0.24348489940166473, 0.2615877091884613]
🧪 Creating normalization transform...
📥 Downloading/loading CIFAR-10 datasets...
Files already downloaded and verified
Files already downloaded and verified
✅ Loaded training samples: 50000, test samples: 10000
🔀 Splitting dataset with ratio 0.80...
✅ Train size: 40000, Validation size: 10000
📦 Creating data loaders with batch size 256...
✅ Data loaders ready.


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

input_shape, num_classes = get_dataset_info(train_dataset)

model_cls = CIFAR10Classifier(
    name="torch_1024_5",
    input_shape=input_shape,
    num_classes=num_classes,
    hidden_layers=[1024,512,256,128,64],
    dropout_rate=[0.3,0.3,0.3,0.3,0.3],
    activation_fn_name="ReLU",
    device=device
    )
model_cls.build_model()
model_cls.compile()
model_cls.summary()

🔍 Extracting dataset info...
✅ Input shape: torch.Size([3, 32, 32]), Number of classes: 10


Layer (type:depth-idx)                   Output Shape              Param #
CIFAR10_torch                            [1, 10]                   --
├─Flatten: 1-1                           [1, 3072]                 --
├─Sequential: 1-2                        [1, 64]                   --
│    └─Linear: 2-1                       [1, 1024]                 3,146,752
│    └─BatchNorm1d: 2-2                  [1, 1024]                 2,048
│    └─ReLU: 2-3                         [1, 1024]                 --
│    └─Dropout: 2-4                      [1, 1024]                 --
│    └─Linear: 2-5                       [1, 512]                  524,800
│    └─BatchNorm1d: 2-6                  [1, 512]                  1,024
│    └─ReLU: 2-7                         [1, 512]                  --
│    └─Dropout: 2-8                      [1, 512]                  --
│    └─Linear: 2-9                       [1, 256]                  131,328
│    └─BatchNorm1d: 2-10                 [1, 256]             

In [6]:
model_cls.train(
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=10,
    verbose=True,
    log_tensorboard=True
)


🚀 Training configuration:
📦 Model name:        torch_1024_5
📐 Input shape:       torch.Size([3, 32, 32])
🔢 Hidden layers:     [1024, 512, 256, 128, 64]
🎛 Dropout rates:     [0.3, 0.3, 0.3, 0.3, 0.3]
⚙️ Activation:        ReLU
📈 Optimizer:         Adam {'lr': 0.01}
🎯 Criterion:         CrossEntropyLoss {}
🧠 Device:            cuda
📊 Epochs:            10
🪄 Early stopping:    True (patience=10)

[1/10] Train loss: 1.8749, acc: 0.3171 | Val loss: 1.6738, acc: 0.3940 | 🕒 21.31s
[2/10] Train loss: 1.6860, acc: 0.3957 | Val loss: 1.5758, acc: 0.4336 | 🕒 20.88s
[3/10] Train loss: 1.5857, acc: 0.4358 | Val loss: 1.5006, acc: 0.4659 | 🕒 20.79s
[4/10] Train loss: 1.5204, acc: 0.4602 | Val loss: 1.4702, acc: 0.4835 | 🕒 20.90s
[5/10] Train loss: 1.4647, acc: 0.4827 | Val loss: 1.4436, acc: 0.4830 | 🕒 21.16s
[6/10] Train loss: 1.4199, acc: 0.4996 | Val loss: 1.4225, acc: 0.4927 | 🕒 20.76s
[7/10] Train loss: 1.3840, acc: 0.5134 | Val loss: 1.3832, acc: 0.5122 | 🕒 20.69s
[8/10] Train loss: 1.3454, a